In [1]:
import pandas as pd
import time
from datetime import datetime

In [2]:
html = 'http://schiphol.dutchplanespotters.nl/?date=2018-09-12'
base_url = 'http://schiphol.dutchplanespotters.nl/?date='
url_contents = pd.read_html(html)

In [84]:
table = url_contents[0]
date_range = pd.date_range('2017-01-01 00:00:00', '2018-12-31 00:00:00', freq='D').strftime('%Y-%m-%d')
print(type(pd.date_range('2017-01-01 00:00:00', '2018-12-31 00:00:00', freq='D')))
    
# Get number of rows and current date
num_rows = len(table)
curr_date = date_range[0]

date_column = pd.Series([curr_date] * num_rows)
table['date'] = date_column
table

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,Aircraft,Arrival,Departure,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,Unnamed: 10_level_0,Unnamed: 11_level_0,Unnamed: 12_level_0,Unnamed: 13_level_0,Unnamed: 14_level_0,Unnamed: 15_level_0,date
,CARGO,AIRCRAFT,AIRLINE,REG,REMARK,GATE,FLIGHTNR,PLACE FROM,STA,ETA,Status,FLIGHTNR,PLACE TO,STD,ETD,Status,
0,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HSC,NaN,C16,HV 6868,Athens/Eleftherios Venizelos (ATH / LGAV),23:50 (11-09-18),00:01,Landed,HV 5695,Samos Island/Samos (SMI / LGSM),05:55,05:56,Departed,2017-01-01
1,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HSJ,NaN,C11,HV 1582,Kefallinia Island/Kefallinia (EFL / LGKF),00:15,00:04,Landed,HV 1621,Heraklion Nikos Kazantzakis (HER / LGIR),06:30,06:28,Departed,2017-01-01
2,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HXM,NaN,C04,HV 5194,Paris-Orly (ORY / LFPO),23:00 (11-09-18),00:05,Landed,HV 6115,Málaga (AGP / LEMG),06:55,07:19,Departed,2017-01-01
3,NaN,Embraer 190 (IGW),KLM,PH-EZB,NaN,B32,KL 1434,Birmingham (BHX / EGBB),21:55 (11-09-18),00:08,Landed,KL 1165,Helsinki Vantaa (HEL / EFHK),07:00,06:57,Departed,2017-01-01
4,NaN,DASA/DORNIER 228-100,Nederlandse Kustwacht,NaN,NaN,NaN,NCG 1,Amsterdam Schiphol (AMS / EHAM),00:19,00:11,Airborne,NCG 1,Amsterdam Schiphol (AMS / EHAM),00:30,00:30,Scheduled,2017-01-01
5,NaN,BOEING 737-700 Winglets,Transavia,PH-XRC,NaN,C08,HV 5642,"Sta Cruz de la Palma, La Palma Island/La Palma...",00:25,00:14,Landed,HV 6411,Nápoli/Naples (NAP / LIRN),06:15,06:13,Departed,2017-01-01
6,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HZL,NaN,D83,HV 650,Antalya (AYT / LTAI),00:20,00:18,Landed,HV 6917,Olbia Costa Smeralda (OLB / LIEO),06:40,06:45,Departed,2017-01-01
7,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HZG,NaN,C09,HV 6118,Málaga (AGP / LEMG),00:05,00:21,Landed,HV 6001,Porto/Francisco de Sá Carneiro (OPO / LPPR),06:15,06:20,Departed,2017-01-01
8,NaN,BOEING 737-800 WINGLETS,Transavia,PH-HXG,NaN,D71,HV 5080,Rodes Island/Diagoras (RHO / LGRP),00:25,00:25,Landed,HV 1205,Zakynthos Island/Dionysios Solomos (ZTH / LGZA),05:25,05:24,Departed,2017-01-01


In [5]:
urls = [base_url + date for date in date_range]
urls

['http://schiphol.dutchplanespotters.nl/?date=2017-01-01',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-02',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-03',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-04',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-05',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-06',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-07',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-08',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-09',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-10',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-11',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-12',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-13',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-14',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-15',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-16',
 'http://schiphol.dutchplanespotters.nl/?date=2017-01-17

In [121]:
df = pd.DataFrame()
column_names = ['Cargo', 'Aircraft', 'Airline', 'Registration', 'Remark', 'Gate', 'Arr_FlightNr', 'Origin', 'STA', 'ETA', 'Status', 'Dep_FlightNr', 'Destination', 'STD', 'ETD', 'Status', 'Date']
for i, url in enumerate(urls):
    # Pull info from url
    day_flights = pd.read_html(url)[0]
    
    # Get number of rows and current date
    num_rows = len(day_flights)
    curr_date = date_range[i]
    
    date_column = pd.Series([curr_date] * num_rows)
    day_flights['date'] = date_column
    
    df = df.append(day_flights)
    time.sleep(1)
    print('Request number: ', i)
    

Request number:  0
Request number:  1
Request number:  2


KeyboardInterrupt: 

In [122]:
df.columns = column_names
df2 = df

In [128]:
def add_date_to_time(col_name):
    df_copy = df.copy()
    mask = df_copy[col_name].str.contains('\d{2}:\d{2}\Z').fillna(False)

    df_copy['Date'] = pd.to_datetime(df_copy['Date'], format='%Y-%m-%d')
    date_string = df_copy['Date'].copy()
    date_string = date_string.loc[mask]
    date_string = pd.DatetimeIndex(date_string)
    date_string = date_string.strftime('(%d-%m-%y)')
    date_string

    df_copy.loc[mask, col_name] = df_copy.loc[mask, col_name].str.cat(date_string, sep =' ')
    return df_copy

df['STA'] = add_date_to_time('STA')['STA']
df['ETA'] = add_date_to_time('ETA')['ETA']
df['STD'] = add_date_to_time('STD')['STD']
df['ETD'] = add_date_to_time('ETD')['ETD']
df

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: A future version of pandas will perform index alignment when `others` is a Series/Index/DataFrame (or a list-like containing one). To disable alignment (the behavior before v.0.23) and silence this warning, use `.values` on any Series/Index/DataFrame in `others`. To enable alignment and silence this warning, pass `join='left'|'outer'|'inner'|'right'`. The future default will be `join='left'`.
  if sys.path[0] == '':


,Cargo,Aircraft,Departure,Registration,Remark,Gate,Arr_FlightNr,Origin,STA,ETA,Status,Dep_FlightNr,Destination,STD,ETD,Status,Date
0,NaN,Boeing 787-8,OR,PH-TFM,NaN,G05,OR 630,Tenerife Island/Tenerife South (TFS / GCTS),23:30 (31-12-16),00:09 (01-01-17),Arrived,OR 337,Cancún (CUN / MMUN),08:00 (01-01-17),08:01 (01-01-17),Departed,2017-01-01
1,NaN,B737-800 Wing,HV,PH-HSC,NaN,D51,HV 6150,Alicante (ALC / LEAL),00:15 (01-01-17),00:11 (01-01-17),Arrived,HV 5749,Casablanca/Mohammed V (CMN / GMMN),06:25 (01-01-17),06:38 (01-01-17),Departed,2017-01-01
2,NaN,B737-800 Wing,HV,PH-HSI,Peter Pan Vakantieclub c/s,D86,HV 552,Marsa Alam (RMF / HEMA),00:30 (01-01-17),00:18 (01-01-17),Arrived,HV 6705,Fuerteventura Island/Fuerteventura (FUE / GCFV),06:10 (02-01-17),06:11 (02-01-17),Departed,2017-01-01
3,NaN,B737-800 Wing,HV,PH-HXC,NaN,C08,HV 6672,Tenerife Island/Tenerife South (TFS / GCTS),00:10 (01-01-17),00:20 (01-01-17),Expected landing,HV 6705,Fuerteventura Island/Fuerteventura (FUE / GCFV),06:05 (01-01-17),06:01 (01-01-17),Departed,2017-01-01
4,NaN,B737-800 Wing,HV,PH-HSA,NaN,D82,HV 5684,Lanzarote Island/Lanzarote (ACE / GCRR),01:00 (01-01-17),01:01 (01-01-17),Arrived,HV 5231,Lanzarote Island/Lanzarote (ACE / GCRR),14:50 (01-01-17),14:58 (01-01-17),Departed,2017-01-01
5,NaN,B737-800 Wing,HV,PH-HSM,NaN,D73,HV 904,Marsa Alam (RMF / HEMA),00:50 (01-01-17),01:36 (01-01-17),Expected landing,HV 5685,Lanzarote Island/Lanzarote (ACE / GCRR),06:00 (01-01-17),05:57 (01-01-17),Departed,2017-01-01
6,NaN,B737-800 Wing,HV,PH-HSK,NaN,D05,HV 6846,Espargos/Amílcar Cabral (SID / GVAC),02:30 (01-01-17),02:25 (01-01-17),Arrived,HV 583,Hurghada (HRG / HEGN),14:10 (01-01-17),14:10 (01-01-17),Departed,2017-01-01
7,NaN,B737-800 Wing,HV,PH-HSE,NaN,C11,HV 6456,Gran Canaria Island/Gran Canaria (LPA / GCLP),03:05 (01-01-17),02:49 (01-01-17),Arrived,HV 6115,Málaga (AGP / LEMG),06:50 (01-01-17),06:59 (01-01-17),Departed,2017-01-01
8,NaN,B737-700,HV,OK-SWW,4th visit*,NaN,HV 5666,Gran Canaria Island/Gran Canaria (LPA / GCLP),00:20 (01-01-17),03:42 (01-01-17),Expected landing,NaN,NaN,NaN,NaN,NaN,2017-01-01
9,NaN,B747-400mc,KL,PH-BFW,NaN,NaN,KL 856,Seoul/Incheon (ICN / RKSI),04:30 (01-01-17),04:06 (01-01-17),Arrived,NaN,NaN,NaN,NaN,NaN,2017-01-01
